### Imports and Utility Variables

In [1]:
%run ../imports.py

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv('./data/play_by_play.csv', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135698 entries, 0 to 1135697
Data columns (total 23 columns):
SCORE                    295258 non-null object
HOMEDESCRIPTION          587421 non-null object
VISITORDESCRIPTION       580417 non-null object
EVENTNUM                 1135698 non-null int64
GAME_ID                  1135698 non-null int64
PCTIMESTRING             1135698 non-null object
PERIOD                   1135698 non-null int64
PLAYER1_ID               1135698 non-null int64
PLAYER1_NAME             1034876 non-null object
PLAYER2_ID               1135698 non-null int64
PLAYER2_NAME             361816 non-null object
PLAYER3_ID               1135698 non-null int64
PLAYER3_NAME             27941 non-null object
SCOREMARGIN              295258 non-null object
Foul                     102457 non-null object
Fouler                   99820 non-null object
Fouler_team              99820 non-null object
is_home_team             102342 non-null object
Foulee                  

In [6]:
df['PERIOD'].value_counts()
# disregarding rare 3 and 4 OT games (periods 7 and 8) and group 1 and 2 OT games together

4    294283
2    289582
3    273457
1    268565
5      8549
6      1035
8       128
7        99
Name: PERIOD, dtype: int64

In [7]:
df_usage = pd.read_csv('./data/usage.csv', compression='gzip')
df_usage .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Columns: 2464 entries, Jabari Parker_AdjUsage to Joe Harris_5-10ft
dtypes: float64(2464)
memory usage: 46.2 MB


In [3]:
# All types of fouls with 100+ occurences over last 3 seasons, raw as written in play-by-play files
# Other rare types listed on: https://pudding.cool/2017/02/two-minute-report/
# (ex: C.P.Foul or Clear Path)
foul_types_raw_all = ['Personal','P.FOUL',
                      'Turnover: Shot Clock',
                      'S.FOUL','Shooting',
                      'L.B.FOUL',
                      'OFF.Foul',
                      'Traveling']

foul_types = ['Personal', '24 Second', 'Shooting', 'Loose Ball', 'Offensive', 'Traveling']

# Originally included more foul types, but processing a season's data took more than 4 hours
foul_types_dict = {'P.FOUL':'Personal', # 'Personal Take Foul':'Personal',
                   'Personal':'Personal', # 'Personal Block':'Personal',
                   'Turnover: Shot Clock':'24 Second',
                   'S.FOUL':'Shooting','Shooting':'Shooting',
                   'L.B.FOUL':'Loose Ball',
                   'OFF.Foul':'Offensive',
                   'Traveling':'Traveling'
                   #'T.Foul':'Technical','T.Foul (Def. 3 Sec':'Defensive 3 Second'
}

features = ['SCORE','HOMEDESCRIPTION','VISITORDESCRIPTION','EVENTNUM','GAME_ID','PCTIMESTRING',
            'PERIOD','PLAYER1_ID','PLAYER1_NAME','PLAYER1_TEAM_NICKNAME',
            'PLAYER2_ID','PLAYER2_NAME','PLAYER2_TEAM_NICKNAME',
            'PLAYER3_ID','PLAYER3_NAME','PLAYER3_TEAM_NICKNAME',
            'SCOREMARGIN'
           ]

team_to_abrev_dict = {'76ers':'PHI',
'Bucks':'MIL',
'Bulls':'CHI',
'Cavaliers':'CLE',
'Celtics':'BOS',
'Clippers':'LAC',
'Grizzlies':'MEM',
'Hawks':'ATL',
'Heat':'MIA',
'Hornets':'CHA',
'Jazz':'UTA',
'Kings':'SAC',
'Knicks':'NYK',
'Lakers':'LAL',
'Magic':'ORL',
'Mavericks':'DAL',
'Nets':'BKN',
'Nuggets':'DEN',
'Pacers':'IND',
'Pelicans':'NOP',
'Pistons':'DET',
'Raptors':'TOR',
'Rockets':'HOU',
'Spurs':'SAS',
'Suns':'PHX',
'Thunder':'OKC',
'Timberwolves':'MIN',
'Trail Blazers':'POR',
'Warriors':'GSW',
'Wizards':'WAS'}

In [10]:
df.head()

,SCORE,HOMEDESCRIPTION,VISITORDESCRIPTION,EVENTNUM,GAME_ID,PCTIMESTRING,PERIOD,PLAYER1_ID,PLAYER1_NAME,PLAYER2_ID,PLAYER2_NAME,PLAYER3_ID,PLAYER3_NAME,SCOREMARGIN,Foul,Fouler,Fouler_team,is_home_team,Foulee,Foulee_team,PLAYER1_TEAM_NICKNAME,PLAYER2_TEAM_NICKNAME,PLAYER3_TEAM_NICKNAME
0,NaN,NaN,NaN,0,21500758,12:00,1,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Jump Ball Gobert vs. Monroe: Tip to Middleton,NaN,1,21500758,12:00,1,203497,Rudy Gobert,202328,Greg Monroe,203114,Khris Middleton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,Bucks,Bucks
2,NaN,NaN,MISS Parker 17' Pullup Jump Shot,2,21500758,11:45,1,203953,Jabari Parker,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bucks,NaN,NaN
3,NaN,Neto REBOUND (Off:0 Def:1),NaN,3,21500758,11:44,1,203526,Raul Neto,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN
4,0 - 2,Neto 4' Fadeaway Jumper (2 PTS),NaN,4,21500758,11:19,1,203526,Raul Neto,0,NaN,0,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN


In [8]:
# In play-by-play csv's, visitor team score is always listed first then home team score
def make_score_cols(df):
    df['Home_score'] = df['SCORE'].map(home_score)
    df['Away_score'] = df['SCORE'].map(away_score)
    
def home_score(score_text):
    try:
        lis = score_text.split(' ') # example: 97 - 95 -> ['97', '-', '95']
        return int(lis[0])
    except: return 0

def away_score(score_text):
    try:
        lis = score_text.split(' ')
        return int(lis[2])
    except: return 0
            
def make_home_away(df):   
    c = 0
    game_id = 0
    for idx, row in df.iterrows():
        c += 1
        if c % 50000 == 0: print(c, ' rows processed')
            
        if (row['GAME_ID'] != game_id) & ('Jump Ball' in str(row['HOMEDESCRIPTION'])):
            game_id = row['GAME_ID']
            if type(row['HOMEDESCRIPTION']) == float: # then text is in visitor column
                away_team = row['PLAYER1_TEAM_NICKNAME']
                df.loc[idx,'Away_team'] = away_team
                df.loc[idx,'Home_team'] = row['PLAYER2_TEAM_NICKNAME']
            elif type(row['VISITORDESCRIPTION']) == float:
                home_team = row['PLAYER1_TEAM_NICKNAME']
                df.loc[idx,'Home_team'] = home_team
                df.loc[idx,'Away_team'] = row['PLAYER2_TEAM_NICKNAME']


In [13]:
temp = df[['GAME_ID','Home_team','Away_team']].groupby('GAME_ID').first()
temp.head()

,Home_team,Away_team
GAME_ID,,
21500001,Hawks,Pistons
21500002,Bulls,Cavaliers
21500003,Warriors,Pelicans
21500004,Magic,Wizards
21500005,Celtics,76ers


In [21]:
groupby_score = df[['GAME_ID','Home_score','Away_score']].groupby('GAME_ID').last()
groupby_score.head()

,Home_score,Away_score
GAME_ID,,
21500001,106,94
21500002,95,97
21500003,95,111
21500004,88,87
21500005,95,112


In [32]:
def make_minute(pctimestring):
    lis = pctimestring.split(':')
    minute, second = int(lis[0]), int(lis[1])
    return minute

def make_second(pctimestring):
    lis = pctimestring.split(':')
    second = int(lis[1])
    return second

def make_time_cols(df):
    df['Minute'] = df['PCTIMESTRING'].map(make_minute)
    df['Second'] = df['PCTIMESTRING'].map(make_second)

def make_home_team(df):
    df['Home_team'] = df['GAME_ID'].map(lambda idx : temp.loc[idx, 'Home_team'])

def make_away_team(df):
    df['Away_team'] = df['GAME_ID'].map(lambda idx : temp.loc[idx, 'Away_team'])
    
def make_score_label(df):
    df['Score_label'] = df['GAME_ID'].map(score_label)

def score_label(game_id):
    str1 = str(temp.loc[game_id, 'Away_team']) + str(groupby_score.loc[game_id, 'Away_score'])
    str2 = str(temp.loc[game_id, 'Home_team']) + str(groupby_score.loc[game_id, 'Home_score'])
    output = str1 + '_' + str2
    return output.replace(' ', '')

In [9]:
make_home_away(df)

50000  rows processed
100000  rows processed
150000  rows processed
200000  rows processed
250000  rows processed
300000  rows processed
350000  rows processed
400000  rows processed
450000  rows processed
500000  rows processed
550000  rows processed
600000  rows processed
650000  rows processed
700000  rows processed
750000  rows processed
800000  rows processed
850000  rows processed
900000  rows processed
950000  rows processed
1000000  rows processed
1050000  rows processed
1100000  rows processed


In [19]:
make_home_team(df)
make_away_team(df)

In [6]:
make_score_cols(df)

In [7]:
make_time_cols(df)

In [33]:
make_score_label(df)

In [34]:
df['Score_label'].value_counts()

Wizards114_Hawks99                 930
Celtics95_Heat105                  898
Pistons144_Bulls147                707
Knicks142_Hawks139                 691
Thunder136_Magic139                640
Rockets127_Warriors132             621
Warriors119_Celtics124             616
Clippers127_Nets122                614
Hornets128_Kings129                597
Bucks109_Hawks117                  593
Grizzlies99_76ers104               592
Magic130_Heat136                   583
Cavaliers108_Hawks110              582
76ers112_Magic109                  581
Hawks104_TrailBlazers109           573
TrailBlazers113_Nuggets115         572
Suns113_Thunder110                 572
Clippers139_Celtics134             570
Pacers122_Jazz119                  570
76ers119_Knicks113                 570
TrailBlazers102_Clippers87         567
Knicks102_Hawks98                  565
Pelicans114_Clippers111            564
Knicks113_Suns111                  563
Rockets140_Clippers132             561
Rockets100_Suns116       

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135698 entries, 0 to 1135697
Data columns (total 30 columns):
SCORE                    295258 non-null object
HOMEDESCRIPTION          587421 non-null object
VISITORDESCRIPTION       580417 non-null object
EVENTNUM                 1135698 non-null int64
GAME_ID                  1135698 non-null int64
PCTIMESTRING             1135698 non-null object
PERIOD                   1135698 non-null int64
PLAYER1_ID               1135698 non-null int64
PLAYER1_NAME             1034876 non-null object
PLAYER2_ID               1135698 non-null int64
PLAYER2_NAME             361816 non-null object
PLAYER3_ID               1135698 non-null int64
PLAYER3_NAME             27941 non-null object
SCOREMARGIN              295258 non-null object
Foul                     102457 non-null object
Fouler                   99820 non-null object
Fouler_team              99820 non-null object
is_home_team             102342 non-null object
Foulee                  

In [35]:
df.to_csv('./data/play_by_play.csv', index=False, compression='gzip')